In [33]:
%cd ..
%load_ext autoreload
%autoreload 2

RTS_DATA = "/mnt/d/rts/"
METADATA = RTS_DATA + 'metadata'
VIDEOS = RTS_DATA + '0'
REMOTE_VIDEOS = "/mnt/rts"
OUTDIR = 'data'



/home/kikohs/work
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from datetime import datetime
import meilisearch
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple

In [34]:
# LOCAL imports
import rts
from rts.ingest import read_video_folder_index

LOG = rts.utils.get_logger()

In [5]:
vidx = read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))

In [3]:
client = meilisearch.Client('http://localhost:7700', '1234')
# client.get_keys()
 
# int(datetime.now().timestamp())
# client.create_index('books', {'primaryKey': 'reference_number'})

NameError: name 'meilisearch' is not defined

In [28]:
def parse_item_sequences(data: Dict, media: Dict) -> Dict:
    seqs = {}
    for id in data.get('idSequence', []):
        seq = {
            'guid': media['guid'],
            'mediaId': media['mediaId'],
            'seqid': id,
            'geo': [],
            'mat': [],
            'pp': []
        }
        seqs[seq['seqid']] = seq

    geo = data.get('VisuelsGEOSequence', [])
    for g in geo:
        seqid, loc = g.split('@')
        seqs[seqid]['geo'].append(loc.lower())

    mat = data.get('VisuelsMATSequence', [])
    for m in mat:
        seqid, mat = m.split('@')
        seqs[seqid]['mat'].append(mat.lower())
    
    pp = data.get('VisuelsPPSequence', [])
    for p in pp:
        seqid, name = p.split('@')
        seqs[seqid]['pp'].append(name.lower())
    
    return seqs


def parse_item(raw: Dict, vidx: Dict) -> Optional[Dict]:
    def find_media_id(support: Dict) -> Tuple[Optional[int], Optional[str]]:
        for i, d in enumerate(support):
            if d.startswith('Z'):
                return i, d
        return 0, None

    try:
        d = raw['response']['docs'][0]
    except IndexError as e:
        LOG.error(e, raw)
        return None
    
    try:
        seq_idx, media_id = find_media_id(d['idSupport'])
    except KeyError as e:
        # LOG.error(e)
        return None
    
    if not media_id:
        return None

    duration = d['DureeMediaSec'][seq_idx]
    ratio = d['RatioMedia'][seq_idx]
    mediaFolderPath = vidx.get(media_id)

    if not mediaFolderPath:
        return None

    media = {
        'guid': d['Guid'],
        'mediaId': media_id,
        'mediaFolderPath': vidx.get(media_id),
        'mediaDuration': duration,
        'ratio': ratio,
        'formatType': d['FormatMedia'][seq_idx],
        'formatResolution': d['DefinitionMedia'][seq_idx],
        'materialType': d['TypeMaterielMedia'][seq_idx],
        'publishedDate': d.get('DatePublication'),
        'publishedBy': d.get('CanalPublication'),

        'rights': d.get('SemaphoreDroit'),
        'categoryName': d.get('CategorieAsset'),
        'assetType': d.get('TypeAsset'),
        'contentType': d.get('TypeContenu'),
        'backgoundType': d.get('idTypeBackground'),
        'collection': d.get('Collection'),
        
        'title': d.get('Titre'),
        'resume': d.get('Resume'),
        'geoTheme': d.get('ThematiquesGEO'),
        'resumeSequence': d.get('ResumeSequence'),
        'created': int(datetime.now().timestamp()),
        'published': int(datetime.strptime(d.get('DatePublication'), "%Y-%m-%dT%H:%M:%S%z").timestamp())
    }

    seqs = parse_item_sequences(d, media)
    media['sequences'] = seqs

    return media


def read_txt_transcript(srtz: zipfile.Path, media: Dict) -> Optional[Dict]:
    try:
        ts = {
            'guid': media['guid'],
            'mediaId': media['mediaId'],
            'version': 1,
            'text': ''
        }
        path = media['guid'] + '_0.txt'
        current = srtz.joinpath(path)
        with current.open('r') as fp:
            wrap = io.TextIOWrapper(fp, encoding='latin-1')
            ts['text'] = wrap.read()
    except Exception as e:
        LOG.error('Read txt ts:', e)
        return None
    return ts


def read_metadata_zippart(root_dir: str, part_num: int, vidx: Dict) -> Dict:
    path = os.path.join(root_dir, f'{part_num}.zip')
    medias = {}
    with zipfile.ZipFile(path, 'r') as z:
        jsons = zipfile.Path(z, f'{part_num}/jsonData/')
        for j in jsons.iterdir():
            with j.open('r') as fp:
                js = orjson.loads(fp.read())
                media = parse_item(js, vidx)
                if media:
                # ts = read_txt_transcript(srts, media)
                # media['ts'] = ts
                    medias[media['guid']] = media

    return medias


def read_all_metadata(root_dir: str, vidx: Dict) -> Dict:
    all_d = dict()
    for i in range(0, 10):
        d = read_metadata_zippart(root_dir, i, vidx)
        all_d.update(d)
    
    return all_d


medias = read_all_metadata(METADATA, vidx)

ERROR:RTS:list index out of range
ERROR:RTS:list index out of range


In [29]:
len(medias)

522103

In [30]:
import pickle

In [32]:
with open('/home/kikohs/work/rts/data/data.pickle', 'wb') as fp:
    pickle.dump(medias, fp)

In [35]:
m2 = rts.utils.dict_from_pickle('/home/kikohs/work/rts/data/data.pickle')

In [3]:
import whisper

model = whisper.load_model("base")
# result = model.transcribe("audio.mp3")
# print(result["text"])

In [36]:
len(m2)

522103

In [ ]:
a = m2.values()